In [36]:
import torch
import torchvision
import numpy as np
import pandas as pd
import seaborn as sn
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision.transforms as transforms
from functools import reduce
import operator

from torch.autograd import Variable
from sklearn.metrics import confusion_matrix

In [2]:
#Data Loading
train = pd.read_csv('mnist/mnist_train.csv')
test = pd.read_csv('mnist/mnist_test.csv')

In [21]:
#Validation Set

#Validation contains 200 Samples
df_train = np.array(train)
df_validation = df_train[:200, :]

#Remove Samples from train set
df_train = df_train[200:, :] 

In [22]:
#Data Processing

#Validation
y_validation = np.reshape(df_validation[:, 0], (len(df_validation), 1))
df_validation = df_validation[:, 1:]
x_validation = df_validation.reshape((200, 1, 28, 28))

#Train/Test
sample_fraction = 10
test_fraction = 10
n_samples = int(df_train.shape[0]/sample_fraction)
df_train = df_train[:n_samples, :]

# Create x_train, y_train with adequate sizes
x_train = df_train[:, 1:] / 255
x_train = np.reshape(x_train, (n_samples, 1, 28, 28))
y_train = np.reshape(df_train[:n_samples, 0], (n_samples, 1))

df_test = np.array(test)
df_test = df_test[:int(n_samples), :]

x_test = df_test[:, 1:] / 255
x_test = np.reshape(x_test, (x_test.shape[0], 1, 28, 28))
y_test = np.reshape(df_test[:, 0], (len(df_test), 1))

#Convert sets to tensor
x_train = torch.FloatTensor(x_train)
y_train = torch.from_numpy(y_train)

x_validation = torch.FloatTensor(x_validation)
y_validation = torch.from_numpy(y_validation)

x_test = torch.FloatTensor(x_test)
y_test = torch.from_numpy(y_test)

print('Number of train samples: {}'.format(n_samples))
x_train.shape, y_train.shape, x_validation.shape, y_validation.shape

Number of train samples: 5979


(torch.Size([5979, 1, 28, 28]),
 torch.Size([5979, 1]),
 torch.Size([200, 1, 28, 28]),
 torch.Size([200, 1]))

In [23]:
batch_size = 2
epochs = 10
train_set = torch.utils.data.TensorDataset(x_train, y_train)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

validation_set = torch.utils.data.TensorDataset(x_validation, y_validation)
validation_loader = torch.utils.data.DataLoader(validation_set, batch_size=batch_size, shuffle=False)

In [42]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #Convolution Layers
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(p=0.25)
        
        #Linear layers
        self.fc1 = nn.Linear(320, 120)
        self.fc1_drop = nn.Dropout(p=0.25)
        self.fc2 = nn.Linear(120, 10)
    
    def forward(self, x):
        x = F.relu(F.max_pool2d(input=self.conv1(x), kernel_size=2))
        x = F.relu(F.max_pool2d(input=self.conv2_drop(self.conv2(x)), kernel_size=2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = F.relu(self.fc2(x))
        return F.log_softmax(x)
                   
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        return num_features
net = Net()
net     

Net(
  (conv1): Conv2d(1, 10, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(10, 20, kernel_size=(5, 5), stride=(1, 1))
  (conv2_drop): Dropout2d(p=0.25)
  (fc1): Linear(in_features=320, out_features=120, bias=True)
  (fc1_drop): Dropout(p=0.25)
  (fc2): Linear(in_features=120, out_features=10, bias=True)
)

In [43]:
device = torch.device('cpu')
net.to(device)

optmizer = optim.Adam(net.parameters())
eq_loss = nn.CrossEntropyLoss()

In [45]:
for epoch in range(epochs):
    total_loss = 0
    net.train()
    for batch, (inputs, targets) in enumerate(train_loader):
        inputs, targets = Variable(inputs).cpu(), Variable(torch.squeeze(targets)).cpu()
        optmizer.zero_grad()
        outputs = net(inputs)
        loss = eq_loss(outputs, targets)
        total_loss += loss
        loss.backward()
        optmizer.step()
    print('Epoch {}\n has Loss {}'.format(epoch, total_loss))
    break
    

torch.Size([2, 1, 28, 28])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([2, 1, 28, 28])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([2, 1, 28, 28])


...

torch.Size([2, 1, 28, 28])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


torch.Size([1, 1, 28, 28])


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


RuntimeError: dimension specified as 0 but tensor has no dimensions

In [ ]:
a = np.zeros(100)
a[50:].shape

In [ ]:
net()
